In [0]:
storageAccountName = "hemzstorage"
storageAccountAccessKey = "<key>"
# sasToken = <sas-token>
blobContainerName = "solutions"
mountPoint = "/mnt/data/"
if not any(mount.mountPoint == mountPoint for mount in dbutils.fs.mounts()):
  try:
    dbutils.fs.mount(
      source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
      mount_point = mountPoint,
      extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
      # extra_configs = {'fs.azure.sas.' + blobContainerName + '.' + storageAccountName + '.blob.core.windows.net': sasToken}
    )
    print("mount succeeded!")
  except Exception as e:
    print("mount exception", e)

In [0]:
dbutils.fs.ls("/mnt/data/input_data")

In [0]:
!pip install -qU langchain langchain-community langchain-huggingface pypdf sentence-transformers

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.2.0 requires pydantic<2,>=1.8.1, but you have pydantic 2.10.3 which is incompatible.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
from langchain_community.document_loaders import (
    PyPDFLoader,
)
from langchain_huggingface import HuggingFaceEmbeddings

In [0]:
loader = PyPDFLoader("/dbfs/mnt/data/input_data/How_to_Buy_Products_Online_1.pdf")
documents = loader.load()

# Extract text from the loaded documents
texts = [doc.page_content for doc in documents]

In [0]:
print(f"The data extracted from the pdf is:\n{documents[0].page_content}")

The data extracted from the pdf is:
How Can I Buy Products From an E-commerce Website?
Step 1: Choose Your Platform
Identify the e-commerce website you want to shop from.
Examples: Amazon, eBay, Walmart.
Step 2: Create an Account
Sign up with your email address and create a password.
Provide accurate details like your name and shipping address.
Step 3: Browse Products
Use the search bar or navigation categories to find items.
Read product descriptions and reviews for informed decisions.
Step 4: Add to Cart
Click "Add to Cart" for items you wish to buy.
Review your cart for accuracy before proceeding.
Step 5: Proceed to Checkout
Click "Checkout" and enter payment details.
Choose payment methods like credit card, PayPal, or others.
Step 6: Confirm Purchase
Review order details and click "Place Order."
Wait for the confirmation email and track your order.


In [0]:
# # Initialize the HuggingFaceEmbeddings instance
# embeddings = HuggingFaceEmbeddings()

# # Generate embeddings for the extracted text
# text_embeddings = [embeddings.embed_documents(text) for text in texts]

2024-12-04 06:36:31.207981: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-04 06:36:31.239328: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/databricks/python/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision

In [0]:
from sentence_transformers import SentenceTransformer

# Load embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
text_embeddings = [model.encode(text) for text in texts]

In [0]:
text_embeddings

[array([ 3.79701108e-02, -1.92569457e-02, -8.37868545e-03, -8.89837183e-03,
        -8.23286269e-03,  6.40797392e-02,  2.55484916e-02,  5.66396229e-02,
        -4.16677706e-02, -3.86959948e-02,  3.53256576e-02, -1.38619747e-02,
         7.66105205e-02, -5.97599521e-02,  6.05619922e-02, -3.59705910e-02,
         1.22235194e-02, -1.61517933e-02,  6.13431388e-04, -2.16930639e-03,
         3.47132236e-02,  3.89006129e-03,  2.81160288e-02,  1.07081216e-02,
        -9.24451724e-02, -4.04665098e-02, -3.61034423e-02,  1.95950214e-02,
        -1.40003012e-02, -3.69353853e-02,  4.59839664e-02, -5.20542189e-02,
         7.58450851e-02,  2.88688950e-02, -5.23324087e-02, -7.66359493e-02,
        -3.87336239e-02, -8.82205516e-02, -7.10645542e-02, -1.38113294e-02,
        -5.71481325e-03, -4.50266227e-02, -1.06380425e-01,  7.49213099e-02,
         7.41631165e-02,  2.90493090e-02,  1.31402677e-02,  8.25693011e-02,
         6.31800815e-02,  4.48988974e-02, -7.29571329e-03,  2.45999601e-02,
        -1.7